In [2]:

#!/usr/bin/env python
# coding: utf-8

# In[153]:


from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import sys, traceback
import json
from urllib import parse


# In[3]:

# entity 들.

KW_CODE =0
KM_CODE =1
DJ_CODE=2
DS_CODE=3
DD_CODE=4
MJ_CODE=5
SY_CODE=6
SM_CODE=7
SK_CODE=8
SW_CODE=9
SS_CODE=10
HS_CODE=11


# In[4]:


base_start_url_list = ['http://kupis.kw.ac.kr/',
'http://lib.kookmin.ac.kr/',
'http://library.daejin.ac.kr/',
'http://discover.duksung.ac.kr/',
'http://library.dongduk.ac.kr/',
'http://lib.mju.ac.kr/',
'http://lib.syu.ac.kr/',
'https://lib.smu.ac.kr/',
'http://library.skuniv.ac.kr/',
'http://lib.swu.ac.kr/',
'http://lib.sungshin.ac.kr/',
'http://hsel.hansung.ac.kr/']

base_middle_url_list = ['search/caz/result?st=KWRD&commandType=advanced&si=1&q=',
'#/total-search?keyword=',
'data_data_list.mir?search_keyword_type1=text&search_keyword1=',
'',
'',
'',
'',
'Search/?q=',
'',
'',
'',
'']





base_end_url_list = ['&b0=and&weight0=&si=2&q=&b1=and&weight1=&si=3&q=&weight2=&inc=TOTAL&_inc=on&_inc=on&_inc=on&_inc=on&_inc=on&_inc=on&lmt0=TOTAL&lmtsn=000000000003&lmtst=OR&rf=&rt=&range=000000000021&cpp=10&msc=100',
'',
'&srch_condi_01=AND&search_keyword_type2=author&search_keyword2=&srch_condi_02=AND&search_keyword_type3=subject&search_keyword3=&srch_condi_03=%E2%96%B2&select_current_sort_type=&select_current_sort_order=&select_current_per_page_num=10&basic_list=list&current_per_page_num=10&current_sort_type=&current_sort_order=&MAXI=1000&mloc_code=DJUL&facet_limit_search_field_code=&scope_code=&mtype_code=M&type_code=',
'',
'',
'',
'',
'',
'',
'',
'',
''
]



class LibraryLS:
    def __init__(self, sid):
        self.sid = sid
        self.loanStatusList = []
        self.errorMessage= ''
    


# In[8]:


class LoanStatus:
    def __init__(self, title = '', RN = '', CN = '', POS = '', STATE = -1, RDD = '', BN = 0, errorMessage = ''):
        self.title = title
        self.RN= RN
        self.CN= CN
        self.POS= POS
        self.STATE= STATE
        self.RDD = RDD#'2222-12-31'
        self.BN = BN
        self.errorMessage = errorMessage
'''#STATE : 도서 상태,
#-1  에러
#1 대출가능
#0 대출중
#2 지정도서
# 대출불가 아직 생각 안함.
#RN : 등록번호
#CN : 청구기호
#POS : 위치
이런것들'''



stateDict = {'대출중': 0, '이용가능':1,'대출가능':1, '지정도서':2}



def findSubStringByRegEx(text, sid, subStringType):
    regex = re.compile(regexDictList[sid][subStringType])
    matchObj = regex.search(text)
    subString = matchObj.group(1)
    return subString
    
    
# 6단계 함수들

# In[10]:
def find_BN(text):
    if '예약' in text:
        regex = re.compile('대출중\s*\(\s+(\d+)명')
        matchObj = regex.search(text)
        num = int(matchObj.group(1))
        return num
    else :
        return 0

# 5단계 함수들

def find_ISBN(text):
    regex = re.compile('\d\d\d\d\d\d\d\d\d\d\d\d\d')
    matchObj = regex.search(text)
    return matchObj.group(0)


def findState(text):
    try:
        STATE = stateDict[text]
    except KeyError:
        STATE = -1
    return STATE

# In[12]:
def find_STATE_BN(text):
    if '대출중' in text:
        return 0, find_BN(text)
    elif '이용가능' in text:
        return 1,0
    elif '지정도서' in text:
        return 2,0
    else :
        return -1,0

def find_RDD(text):
    regex = re.compile('\d\d\d\d\-\d\d\-\d\d')
    matchObj = regex.search(text)
    return matchObj.group(0)


# 4단계 함수들

def isISBNCorrect(ISBN, bsObject, sid):
    try:
        if sid == KW_CODE:
            searchedISBN = find_ISBN(str(bsObject.find('div', {'id': 'divProfile'})))
            if ISBN == searchedISBN:
                return True
            else :
                return False
        elif sid == DJ_CODE:
            pass
        elif sid == DS_CODE:
            pass
        elif sid == DD_CODE:
            pass
        elif sid == MJ_CODE:
            pass
        elif sid == SY_CODE:
            pass
        elif sid == SM_CODE:
            includedISBNString = str(bsObject.find('div', {'class':'col-md-10 detail-table-right'}).find_all('dl')[2].find('dd'))
            if ISBN == find_ISBN(includedISBNString):
                return True
            else :
                return False
        elif sid == SK_CODE:
            pass
        elif sid == SW_CODE:
            pass
        elif sid == SS_CODE:
            pass
        elif sid == HS_CODE:
            pass
    except AttributeError:
        # 전자 자료 같은 경우. ISBN이 없다.
        return False





def makeLoanResultList(bsObject, sid):
    if sid == KW_CODE:
        LoanResultList = bsObject.find('div', {'id': 'divHoldingInfo'}).find('table').find('tbody').find_all('tr')
    elif sid == KM_CODE:
        pass
    elif sid == DJ_CODE:
        pass
    elif sid == DS_CODE:
        pass
    elif sid == DD_CODE:
        pass
    elif sid == MJ_CODE:
        pass
    elif sid == SY_CODE:
        pass
    elif sid == SM_CODE:
        LoanResultList = bsObject.find('div', class_='sponge-guide-Box-table sponge-detail-table').find('tbody').find_all('tr')
    elif sid == SK_CODE:
        pass
    elif sid == SW_CODE:
        pass
    elif sid == SS_CODE:
        pass
    elif sid == HS_CODE:
        pass
    return LoanResultList



def getLoanStatus(title, loanResult, loanStatusList, sid):
    if sid == KW_CODE:
        errorMessage = ''
        content = loanResult.find_all('td')
        RN = content[1].text.strip()
        CN = content[2].text.strip()
        POS = content[3].text.strip()
        STATE_STRING = content[4].text.strip()
        RDD = content[5].text.strip()
        STATE = findState(STATE_STRING)
        BN = 0
    elif sid == KM_CODE:
        pass
    elif sid == DJ_CODE:
        pass
    elif sid == DS_CODE:
        pass
    elif sid == DD_CODE:
        pass
    elif sid == MJ_CODE:
        pass
    elif sid == SY_CODE:
        pass
    elif sid == SM_CODE:
        errorMessage = ''
        content = loanResult.find_all('td')
        RN = content[0].text.strip()
        CN = content[1].text.strip()
        POS = content[2].text.strip()
        STATE_STRING = content[3].text.strip()
        try:
            RDD = find_RDD(str(content[4]))
        except AttributeError as e:
            RDD = ''
        STATE, BN = find_STATE_BN(STATE_STRING)
    elif sid == SK_CODE:
        pass
    elif sid == SW_CODE:
        pass
    elif sid == SS_CODE:
        pass
    elif sid == HS_CODE:
        pass
    
    loanStatusList.append(LoanStatus(title, RN, CN,POS,STATE, RDD ,BN, ''))
    return

# 3단계 함수들

def makeBookResultList(bsObject, sid):
    if sid == KW_CODE:
        BookResultList = bsObject.find('div', {'id' : 'divSearchResult'}).find_all('dl', {'class': 'briefDetail'})
    elif sid == KM_CODE:
        pass
    elif sid == DJ_CODE:
        pass
    elif sid == DS_CODE:
        pass
    elif sid == DD_CODE:
        pass
    elif sid == MJ_CODE:
        pass
    elif sid == SY_CODE:
        pass
    elif sid == SM_CODE:
        BookResultList =  bsObject.find_all('div', class_='search-list-result')
    elif sid == SK_CODE:
        pass
    elif sid == SW_CODE:
        pass
    elif sid == SS_CODE:
        pass
    elif sid == HS_CODE:
        pass
    return BookResultList


def makeContent(bookResult, sid):
    if sid == KW_CODE:
        content = bookResult.find('dd', {'class': 'searchTitle'}).find('a')
    elif sid == KM_CODE:
        pass
    elif sid == DJ_CODE:
        pass
    elif sid == DS_CODE:
        pass
    elif sid == DD_CODE:
        pass
    elif sid == MJ_CODE:
        pass
    elif sid == SY_CODE:
        pass
    elif sid == SM_CODE:
        content =   bookResult.find('div', {'class','sponge-list-content'}).find('a')
    elif sid == SK_CODE:
        pass
    elif sid == SW_CODE:
        pass
    elif sid == SS_CODE:
        pass
    elif sid == HS_CODE:
        pass
    return content


def visitLink(ISBN, sid, title, bookLink):
    loanStatusList = []
    #= LoanStatus()
    try:
        #1. 접속.
        html = urlopen(base_start_url_list[sid]+bookLink)
        bsObject = BeautifulSoup(html, "html.parser")
        
        #1-1 ISBN 정보가 일치하지 않다면 바로 return!!!
        if isISBNCorrect(ISBN, bsObject, sid) == False:
            return loanStatusList

        #2. 대출 데이터를 차곡차곡저장.
        idx = 0
        for loanResult in makeLoanResultList(bsObject, sid):
            getLoanStatus(title, loanResult, loanStatusList, sid)
    except :
        traceback.print_exc()
        #traceback.format_exc()
    finally:
        return loanStatusList


# 2단계 함수들
#아직 상명대, 광운대만 함. 
def makeSearchUrl(ISBN, title, sid, flag):
    if flag ==1:
        searchUrl = base_start_url_list[sid]+base_middle_url_list[sid] + ISBN+ base_end_url_list[sid]
    else :
        # flag 2.
        searchUrl = base_start_url_list[sid]+base_middle_url_list[sid] + title+ base_end_url_list[sid]
    return searchUrl

def crawling(ISBN, title, sid, searchUrl):
    libraryLoanStatus = LibraryLS(sid)
    try:
        #1. 접속.
        html = urlopen(searchUrl)
        bsObject = BeautifulSoup(html, "html.parser")
        #2. 링크를 차곡차곡저장.
        # 정규식과 beautifulsoap4을 이용하여 저장...!!
        bookLinklist =[]
        for bookResult in makeBookResultList(bsObject, sid):
            content = makeContent(bookResult, sid)
            title = content.text.strip()
            link =  content.get('href')
            bookLinklist.append([title, link])

        #3. 해당 링크를 방문하여 리스트에 추가해라..!!
        for item in bookLinklist:
            libraryLoanStatus.loanStatusList += visitLink(ISBN, sid, item[0], item[1])
    except:
        libraryLoanStatus.errorMessage = traceback.format_exc()
        #libraryLoanStatus.errorMessage = 에러 traceback 에러메세지를 담는다.
    finally:
        return libraryLoanStatus



def makeJsonDict(result):
    # result 의 타입은 LibraryLS
    jsonDict = {}
    jsonDict['sid'] = result.sid
    jsonDict['loanStatusList'] = []
    for item in result.loanStatusList:
        jsonDict['loanStatusList'].append(vars(item))
    jsonDict['errorMessage']=result.errorMessage
    return jsonDict


# 1단계 함수

def findLoanStatus(ISBN, title, sid, searchFlag):
    if sid != SM_CODE and sid != KW_CODE:
        result = LibraryLS(sid)
        result.errorMessage = 'Not developed...'
    else : 
        searchUrl = makeSearchUrl(ISBN, title, sid, searchFlag)
        result = crawling(ISBN, title, sid, searchUrl)
    # json string 으로 만들기.
    jsonDict = makeJsonDict(result)
    return json.dumps(jsonDict)
    



#ISBN = '9788988474839'
#koreaTitle = '데이터 분석 전문가 가이드'
#sid = KW_CODE
#title = parse.quote(koreaTitle)
#searchFlag = 2
#resultfinalfinal = findLoanStatus(ISBN, title, sid,searchFlag)
#resultfinalfinal


In [3]:
'''def makeSearchUrl(ISBN, title, sid, flag):
    if flag ==1:
        searchUrl = base_start_url_list[sid]+base_middle_url_list[sid] + ISBN+ base_end_url_list[sid]
    else :
        # flag 2.
        searchUrl = base_start_url_list[sid]+base_middle_url_list[sid] + title+ base_end_url_list[sid]
    return searchUrl'''

'def makeSearchUrl(ISBN, title, sid, flag):\n    if flag ==1:\n        searchUrl = base_start_url_list[sid]+base_middle_url_list[sid] + ISBN+ base_end_url_list[sid]\n    else :\n        # flag 2.\n        searchUrl = base_start_url_list[sid]+base_middle_url_list[sid] + title+ base_end_url_list[sid]\n    return searchUrl'

In [4]:
ISBN = '9788988474839'
koreaTitle = '자존감 수업'
sid = DJ_CODE
title = parse.quote(koreaTitle)
searchFlag = 2
#resultfinalfinal = findLoanStatus(ISBN, title, sid,searchFlag)
#resultfinalfinal

In [5]:
searchUrl = makeSearchUrl(ISBN, title, sid, searchFlag)

In [6]:
searchUrl

'http://library.daejin.ac.kr/data_data_list.mir?search_keyword_type1=text&search_keyword1=%EC%9E%90%EC%A1%B4%EA%B0%90%20%EC%88%98%EC%97%85&srch_condi_01=AND&search_keyword_type2=author&search_keyword2=&srch_condi_02=AND&search_keyword_type3=subject&search_keyword3=&srch_condi_03=%E2%96%B2&select_current_sort_type=&select_current_sort_order=&select_current_per_page_num=10&basic_list=list&current_per_page_num=10&current_sort_type=&current_sort_order=&MAXI=1000&mloc_code=DJUL&facet_limit_search_field_code=&scope_code=&mtype_code=M&type_code='

In [7]:
html = urlopen(searchUrl)
bsObject = BeautifulSoup(html, "html.parser")

In [8]:
bsObject

<!DOCTYPE html>

<html lang="ko">
<head>
<meta charset="utf-8"/> <meta content="IE=edge" http-equiv="X-UA-Compatible"/> <meta content="width=device-width, initial-scale=1" name="viewport"/> <title>소장자료 검색-대진대학교 중앙도서관</title> <script src="/liberty/home/plugin/jquery/js/jquery-1.12.2.min.js" type="text/javascript"></script> <script src="/liberty/home/plugin/jquery/js/jquery-ui.min.js?1.12.0" type="text/javascript"></script> <script src="/liberty/home/plugin/jquery/js/jquery.modal.js" type="text/javascript"></script> <script src="/liberty/home/plugin/jquery/js/jquery.mousewheel.js" type="text/javascript"></script> <script src="/liberty/home/plugin/jquery/js/jquery.cookie.js" type="text/javascript"></script> <script src="/liberty/home/plugin/bootstrap/js/bootstrap.min.js?3.3.6" type="text/javascript"></script> <script src="/liberty/home/plugin/bootstrap/js/bootstrap-slick.min.js?1.6.0" type="text/javascript"></script> <script src="/liberty/home/js/ko/mir_home_common.js" type="text/javascri

In [12]:
len(bsObject.find_all('div', {'class' : 'table_list_default table_list'}))

1

In [14]:
len(bsObject.find('div', {'class' : 'table_list_default table_list'}).find('tbody').find_all('tr'))

16

In [15]:
tr_result = bsObject.find('div', {'class' : 'table_list_default table_list'}).find('tbody').find_all('tr')

In [16]:
BookResultList = []
for i in range(0,len(tr_result)):
    if i%2 ==0 :
        BookResultList.append(tr_result[i])

In [17]:
len(BookResultList)

8

In [18]:
BookResultList

[<tr>
 <td><input class="checkbox_chk" title="자존감 수업 :하루에 하나, 나를 사랑하게 되는 자존감 회복 훈련" type="checkbox" value="384298"/></td>
 <td>
 <span class="book_image">
 <span class="data_search_list_img_loading"><i class="fa fa-spinner fa-spin fa-fw"></i></span>
 <img alt="책 표지" class="media-object" id="img_book" src="" style="display:none;"/>
 </span>
 </td>
 <td class="text-left">
 <a class="book_title" href="#link" onclick="do_search_report('384298','DJUL','');">
 <span class="label label_book_type_BK">도서</span>
 																				자존감 수업 :하루에 하나, 나를 사랑하게 되는 자존감 회복 훈련</a>
 <ul class="board_info">
 <li class="book_author">
 <span>저자 : 윤홍균</span>
 <span>출판사 : 심플라이프</span>
 <span>출판연도 : 2016</span>
 </li>
 <li class="book_site">
 																				청구기호 : 155.2 윤95ㅈ</li>
 <li class="book_date">
 </li>
 <li>
 																				대출상태 : 대출가능</li>
 </ul>
 </td>
 <td class="hidden-xs hidden-sm" style="vertical-align:middle;">
 <button class="btn btn_mir_search_view btn-sm" id="reportview" onclick="sh

In [21]:
res2 =BookResultList[1].find('a', {'class', 'book_title'})

In [22]:
res2

<a class="book_title" href="#link" onclick="do_search_report('934846','DJUL','');">
<span class="label label_book_type_BK">도서</span>
																				이쁘게 나이드는 당신이 좋다 :가족학 박사의 나를 사랑하게 되는 감정수업</a>

In [23]:
res2.get('href')

'#link'